In [1]:
import ezdxf
import subprocess
import ezdxf

read dwg file 

In [9]:
import subprocess
import os
import glob

input_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dwg_file"
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file"
oda_exe = r"C:\Program Files\ODA\ODAFileConverter 26.7.0\ODAFileConverter.exe"

os.makedirs(output_folder, exist_ok=True)

cmd = [
    oda_exe,
    input_folder,
    output_folder,
    "ACAD2013",  # safer than 2010
    "DXF",
    "0",
    "*.dwg"
]

result = subprocess.run(cmd, capture_output=True, text=True)

print("Return code:", result.returncode)
print("STDOUT:\n", result.stdout)
print("STDERR:\n", result.stderr)

dxf_files = glob.glob(os.path.join(output_folder, "*.dxf"))
print("Converted DXF files:", dxf_files)


Return code: 0
STDOUT:
 
STDERR:
 
Converted DXF files: ['D:\\2_Analytics\\6_plan_vs_actual\\raw_data_dwg_file\\dxf_file\\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf']


read dxf file and spilt the feature manually 

In [ ]:
import ezdxf
import os

# Input DXF file
input_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf"

# Output folder
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features"
os.makedirs(output_folder, exist_ok=True)

# Read DXF
doc = ezdxf.readfile(input_dxf)
msp = doc.modelspace()

# Buckets for separated entities
lines = []
polygons = []
points = []
circles = []
arcs = []
texts = []
hatches = []

# Iterate over entities
for entity in msp:
    etype = entity.dxftype()

    if etype == "LINE":
        lines.append(entity)

    elif etype == "POINT":
        points.append(entity)

    elif etype in ["LWPOLYLINE", "POLYLINE"]:
        if entity.closed:   # ✅ closed polyline = polygon
            polygons.append(entity)
        else:
            lines.append(entity)

    elif etype == "CIRCLE":
        polygons.append(entity)  # ✅ treat circle as polygon (closed shape)

    elif etype == "ARC":
        arcs.append(entity)

    elif etype in ["TEXT", "MTEXT"]:
        texts.append(entity)

    elif etype == "HATCH":
        polygons.append(entity)  # ✅ hatch = polygon-like fill

# Function to save entities into new DXF
def save_entities(entities, filename):
    if not entities:
        return
    new_doc = ezdxf.new(setup=True)
    new_msp = new_doc.modelspace()
    for e in entities:
        try:
            new_msp.add_foreign_entity(e)
        except Exception as ex:
            print(f"⚠️ Skipped {e.dxftype()}: {ex}")
    new_doc.saveas(filename)

# Save to separate DXF files
save_entities(lines, os.path.join(output_folder, "LINES.dxf"))
save_entities(polygons, os.path.join(output_folder, "POLYGONS.dxf"))
save_entities(points, os.path.join(output_folder, "POINTS.dxf"))
save_entities(circles, os.path.join(output_folder, "CIRCLES.dxf"))
save_entities(arcs, os.path.join(output_folder, "ARCS.dxf"))
save_entities(texts, os.path.join(output_folder, "TEXTS.dxf"))
save_entities(hatches, os.path.join(output_folder, "HATCHES.dxf"))

print(" DXF files saved in:", output_folder)


✅ DXF files saved in: D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features


READ TEXT AND LINE FILE 

In [22]:
import ezdxf

# Paths to split files
lines_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features\LINES.dxf"
texts_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features\TEXTS.dxf"

# ---- Read Lines (LINE + LWPOLYLINE) ----
print("📌 Reading LINES.dxf...")
doc_lines = ezdxf.readfile(lines_dxf)
msp_lines = doc_lines.modelspace()

line_data = []

# True LINE entities
for line in msp_lines.query("LINE"):
    start = (line.dxf.start.x, line.dxf.start.y, line.dxf.start.z)
    end = (line.dxf.end.x, line.dxf.end.y, line.dxf.end.z)
    line_data.append({"start": start, "end": end})

# LWPOLYLINE entities (can be 2+ vertices)
for pline in msp_lines.query("LWPOLYLINE"):
    points = list(pline.get_points("xy"))  # extract 2D points
    if len(points) == 2:  # only two vertices -> treat as line
        line_data.append({"start": points[0], "end": points[1]})
    else:
        # break into segments
        for i in range(len(points) - 1):
            line_data.append({"start": points[i], "end": points[i + 1]})
        if pline.closed:  # if closed, connect last->first
            line_data.append({"start": points[-1], "end": points[0]})

print(f"Found {len(line_data)} line segments")
if line_data:
    print("First few lines:", line_data[:5])

# ---- Read Texts ----
print("\n📌 Reading TEXTS.dxf...")
doc_texts = ezdxf.readfile(texts_dxf)
msp_texts = doc_texts.modelspace()

text_data = []
for t in msp_texts.query("TEXT MTEXT"):
    content = t.dxf.text
    insert = (t.dxf.insert.x, t.dxf.insert.y, t.dxf.insert.z)
    text_data.append({"text": content, "insert": insert})

print(f"Found {len(text_data)} texts")
if text_data:
    print("First few texts:", text_data[:100])


📌 Reading LINES.dxf...
Found 19 line segments
First few lines: [{'start': (307481.8759446839, 2188842.655843234, -209.8793999999999), 'end': (307982.6560188805, 2190234.929635791, -209.8793999999999)}, {'start': (305868.2291071674, 2189379.171865095, -209.8793999999999), 'end': (306395.7423981508, 2190845.769615494, -209.8793999999999)}, {'start': (307481.8759446839, 2188842.655843234, -209.8793999999999), 'end': (307982.6560188805, 2190234.929635791, -209.8793999999999)}, {'start': (307095.7003059766, 2188654.384147289, 141.6396979186909), 'end': (307784.1841949472, 2190569.511670815, -209.9246924562343)}, {'start': (305604.2312609017, 2189531.577666094, -209.8793999999999), 'end': (306122.5061352671, 2190972.490677813, -209.8793999999999)}]

📌 Reading TEXTS.dxf...
Found 103 texts
First few texts: [{'text': '216.000', 'insert': (308605.2381656518, 2189556.376533808, 0.0)}, {'text': '208.000', 'insert': (308589.0711933865, 2189560.869025644, 0.0)}, {'text': '173.000', 'insert': (307912

remove T from text

In [23]:
# Input and output paths
texts_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features\TEXTS.dxf"
output_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features\TEXTS_cleaned.dxf"

# Read DXF
doc = ezdxf.readfile(texts_dxf)
msp = doc.modelspace()

# Define unwanted texts (exact match or patterns)
unwanted = {"T", "{\\fArial|b0|i0|c0|p34;T}", "{\\fArial|b0|i0|c0|p34;B'}",
            "{\\fArial|b0|i0|c0|p34;C'}", "{\\fArial|b0|i0|c0|p34;D'}",
            "{\\fArial|b0|i0|c0|p34;E'}", "{\\fArial|b0|i0|c0|p34;B}"}

# Collect cleaned texts
kept_texts = []
for t in msp.query("TEXT MTEXT"):
    content = t.dxf.text.strip()
    if content not in unwanted:
        kept_texts.append((t.dxf.insert, content))
    else:
        print(f"🗑️ Removed text: {content}")

# Create new DXF with only allowed texts
doc_new = ezdxf.new(setup=True)
msp_new = doc_new.modelspace()

for insert, content in kept_texts:
    msp_new.add_text(content, dxfattribs={"insert": insert})

# Save
doc_new.saveas(output_dxf)
print(f"Cleaned text DXF saved to: {output_dxf}")
print(f"Kept {len(kept_texts)} texts")


🗑️ Removed text: {\fArial|b0|i0|c0|p34;B'}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;C'}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;D'}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;E'}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;B}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i0|c0|p34;T}
🗑️ Removed text: {\fArial|b0|i

Remove the duplicates

In [ ]:
# Paths
lines_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features\LINES.dxf"
texts_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_features\TEXTS.dxf"

output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\cleaned_features"
os.makedirs(output_folder, exist_ok=True)

# ---------- Deduplicate Lines ----------
doc_lines = ezdxf.readfile(lines_dxf)
msp_lines = doc_lines.modelspace()

line_set = set()

# LINE entities
for line in msp_lines.query("LINE"):
    start = (round(line.dxf.start.x, 6), round(line.dxf.start.y, 6), round(line.dxf.start.z, 6))
    end = (round(line.dxf.end.x, 6), round(line.dxf.end.y, 6), round(line.dxf.end.z, 6))
    # normalize order
    key = tuple(sorted([start, end]))
    line_set.add(key)

# LWPOLYLINE entities
for pline in msp_lines.query("LWPOLYLINE"):
    points = list(pline.get_points("xy"))
    # break into segments
    for i in range(len(points) - 1):
        start = (round(points[i][0], 6), round(points[i][1], 6))
        end = (round(points[i+1][0], 6), round(points[i+1][1], 6))
        key = tuple(sorted([start, end]))
        line_set.add(key)
    if pline.closed and len(points) > 2:
        start = (round(points[-1][0], 6), round(points[-1][1], 6))
        end = (round(points[0][0], 6), round(points[0][1], 6))
        key = tuple(sorted([start, end]))
        line_set.add(key)

print(f" Unique lines found: {len(line_set)}")

# Save deduplicated lines
doc_new_lines = ezdxf.new(setup=True)
msp_new_lines = doc_new_lines.modelspace()
for start, end in line_set:
    msp_new_lines.add_line(start, end)
doc_new_lines.saveas(os.path.join(output_folder, "LINES_clean.dxf"))


# ---------- Deduplicate Texts ----------
doc_texts = ezdxf.readfile(texts_dxf)
msp_texts = doc_texts.modelspace()

text_set = set()
for t in msp_texts.query("TEXT MTEXT"):
    content = t.dxf.text.strip()
    insert = (round(t.dxf.insert.x, 6), round(t.dxf.insert.y, 6), round(t.dxf.insert.z, 6))
    key = (insert, content)
    text_set.add(key)

print(f" Unique texts found: {len(text_set)}")

# Save deduplicated texts
doc_new_texts = ezdxf.new(setup=True)
msp_new_texts = doc_new_texts.modelspace()
for insert, content in text_set:
    msp_new_texts.add_text(content, dxfattribs={"insert": insert})
doc_new_texts.saveas(os.path.join(output_folder, "TEXTS_clean.dxf"))

print(" Cleaned DXFs saved in:", output_folder)


 Unique lines found: 18
 Unique texts found: 103
🎉 Cleaned DXFs saved in: D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\cleaned_features


read the dxf file attribute data

In [ ]:
import ezdxf
import pandas as pd

# Path to your DXF file
dxf_file = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf"

#dxf_file = r"D:\path\to\your.dxf"
doc = ezdxf.readfile(dxf_file)
msp = doc.modelspace()

for entity in msp:
    print(f"\nEntity: {entity.dxftype()}")
    for key, value in entity.dxf.all_existing_dxf_attribs().items():
        print(f"  {key}: {value}")


Entity: HATCH
  handle: 2EB
  owner: 1F
  layer: PRO 2024-25 RL216-208
  ltscale: 2.5
  lineweight: 5
  elevation: (0.0, 0.0, 0.0)
  extrusion: (0.0, 0.0, 1.0)
  pattern_name: ANSI31
  solid_fill: 0
  associative: 0
  hatch_style: 1
  pattern_type: 1
  pattern_angle: 0.0
  pattern_scale: 50.0
  pattern_double: 0

Entity: LWPOLYLINE
  handle: 2EC
  owner: 1F
  layer: PRO 2024-25 RL216-208
  ltscale: 2.5
  lineweight: 5
  flags: 1
  const_width: 2.0
  elevation: 200.571

Entity: HATCH
  handle: 2ED
  owner: 1F
  layer: PRO 2024-25 RL 208-200
  linetype: Continuous
  ltscale: 2.0
  lineweight: 15
  elevation: (0.0, 0.0, 0.0)
  extrusion: (0.0, 0.0, 1.0)
  pattern_name: ANSI31
  solid_fill: 0
  associative: 0
  hatch_style: 1
  pattern_type: 1
  pattern_angle: 0.0
  pattern_scale: 50.0
  pattern_double: 0

Entity: LWPOLYLINE
  handle: 2EE
  owner: 1F
  layer: PRO 2024-25 RL 208-200
  linetype: Continuous
  ltscale: 2.0
  lineweight: 60
  flags: 1
  const_width: 2.0

Entity: HATCH
  handle

In [17]:
import ezdxf
import os
from collections import defaultdict

# Input DXF
input_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf"

# Output folder
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer"
os.makedirs(output_folder, exist_ok=True)

# Load DXF
doc = ezdxf.readfile(input_dxf)
msp = doc.modelspace()

# Group entities by layer name
layer_entities = defaultdict(list)
for entity in msp:
    layer_entities[entity.dxf.layer].append(entity)

# Function to save entities into new DXF
def save_entities(entities, filename):
    if not entities:
        return
    new_doc = ezdxf.new(setup=True)
    new_msp = new_doc.modelspace()
    for e in entities:
        try:
            new_msp.add_foreign_entity(e)
        except Exception as ex:
            print(f"⚠️ Skipped {e.dxftype()}: {ex}")
    new_doc.saveas(filename)

# Save one DXF per layer
for layer, entities in layer_entities.items():
    safe_layer = layer.replace(" ", "_")  # avoid spaces in filenames
    out_file = os.path.join(output_folder, f"{safe_layer}.dxf")
    save_entities(entities, out_file)
    print(f"Layer '{layer}': saved {len(entities)} entities → {out_file}")

print("✅ Done: Split DXF by layer")


Layer 'PRO 2024-25 RL216-208': saved 6 entities → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\PRO_2024-25_RL216-208.dxf
Layer 'PRO 2024-25 RL 208-200': saved 4 entities → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\PRO_2024-25_RL_208-200.dxf
Layer 'PRO 2024-25 RL 200-191': saved 4 entities → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\PRO_2024-25_RL_200-191.dxf
Layer 'PRO 2024-25 RL 191-182': saved 4 entities → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\PRO_2024-25_RL_191-182.dxf
Layer 'PRO 2024-25 RL 182-173': saved 4 entities → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\PRO_2024-25_RL_182-173.dxf
Layer 'PRO 2024-25 RL 173-164': saved 4 entities → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\PRO_2024-25_RL_173-164.dxf
Layer 'PRO 2024-25 RL 164-155': saved 4 entities → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\PRO_2024-25_RL_164-155.dxf
L

In [ ]:
import ezdxf
import os
import csv
from collections import defaultdict

# Input DXF
input_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf"

# Output folder
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_new_2"
os.makedirs(output_folder, exist_ok=True)

# CSV file for summary
csv_file = os.path.join(output_folder, "layer_summary.csv")

# Load DXF
doc = ezdxf.readfile(input_dxf)
msp = doc.modelspace()

# Group entities by layer name
layer_entities = defaultdict(list)
for entity in msp:
    layer_entities[entity.dxf.layer].append(entity)

# Function to save entities into new DXF
def save_entities(entities, filename):
    if not entities:
        return
    new_doc = ezdxf.new(setup=True)
    new_msp = new_doc.modelspace()
    for e in entities:
        try:
            new_msp.add_foreign_entity(e)
        except Exception as ex:
            print(f"⚠️ Skipped {e.dxftype()}: {ex}")
    new_doc.saveas(filename)

# Prepare CSV header
with open(csv_file, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Layer", "EntityType", "Count", "OutputFile"])

    # Save one DXF per layer and record summary
    for layer, entities in layer_entities.items():
        # Count entity types
        type_counts = defaultdict(int)
        for e in entities:
            type_counts[e.dxftype()] += 1

        # Print summary in console
        print(f"\nLayer '{layer}' → {len(entities)} entities")
        for t, count in type_counts.items():
            print(f"   - {t}: {count}")

        # Create safe filename
        safe_layer = layer.replace(" ", "_")
        out_file = os.path.join(output_folder, f"{safe_layer}.dxf")

        # Save DXF file
        save_entities(entities, out_file)
        print(f"   Saved → {out_file}")

        # Write summary rows into CSV
        for t, count in type_counts.items():
            writer.writerow([layer, t, count, out_file])

print("\n Done: Split DXF by layer and created summary CSV")
print(f" Summary saved to: {csv_file}")



📂 Layer 'PRO 2024-25 RL216-208' → 6 entities
   - HATCH: 1
   - LWPOLYLINE: 1
   - TEXT: 4
   ✅ Saved → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_new_2\PRO_2024-25_RL216-208.dxf

📂 Layer 'PRO 2024-25 RL 208-200' → 4 entities
   - HATCH: 1
   - LWPOLYLINE: 1
   - TEXT: 2
   ✅ Saved → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_new_2\PRO_2024-25_RL_208-200.dxf

📂 Layer 'PRO 2024-25 RL 200-191' → 4 entities
   - HATCH: 1
   - LWPOLYLINE: 1
   - TEXT: 2
   ✅ Saved → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_new_2\PRO_2024-25_RL_200-191.dxf

📂 Layer 'PRO 2024-25 RL 191-182' → 4 entities
   - HATCH: 1
   - LWPOLYLINE: 1
   - TEXT: 2
   ✅ Saved → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_new_2\PRO_2024-25_RL_191-182.dxf

📂 Layer 'PRO 2024-25 RL 182-173' → 4 entities
   - HATCH: 1
   - LWPOLYLINE: 1
   - TEXT: 2
   ✅ Saved → D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_new_2\PRO_2024-

In [2]:
import ezdxf
import os
from collections import defaultdict, Counter

# === INPUT / OUTPUT ===
input_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf"
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer"
os.makedirs(output_folder, exist_ok=True)
out_file = os.path.join(output_folder, "section_lines.dxf")

# === RULES ===
# Only these types are allowed in the target "section lines" layer:
ALLOWED_TYPES = {"LINE", "LWPOLYLINE", "TEXT", "MTEXT"}
# Explicitly disallow common polygonal/fill types:
DISALLOWED_TYPES = {"HATCH", "POLYLINE"}  # add more if needed (e.g., "SPLINE", "CIRCLE", ...)

def is_lwpolyline_closed(lp) -> bool:
    """Return True if an LWPOLYLINE is closed."""
    # ezdxf LWPOLYLINE exposes .closed property in recent versions
    closed_attr = getattr(lp, "closed", None)
    if isinstance(closed_attr, bool):
        return closed_attr
    # Fallback: check DXF flags bit 1
    try:
        flags = int(lp.dxf.flags)
        return bool(flags & 1)
    except Exception:
        # If we can't determine, be conservative and treat as closed
        return True

def layer_matches_section_criteria(entities):
    """Check that a layer contains ONLY allowed types, with at least one text and one line-ish entity,
    and that NO LWPOLYLINE is closed (i.e., no polygons)."""
    if not entities:
        return False

    type_counts = Counter(e.dxftype() for e in entities)

    # Hard disallow first
    if any(t in DISALLOWED_TYPES for t in type_counts):
        return False

    # Must be subset of allowed types
    if not set(type_counts).issubset(ALLOWED_TYPES):
        return False

    # Must have some text (section letters)
    text_count = type_counts.get("TEXT", 0) + type_counts.get("MTEXT", 0)
    if text_count == 0:
        return False

    # Must have some lines (LINE or LWPOLYLINE)
    lineish_count = type_counts.get("LINE", 0) + type_counts.get("LWPOLYLINE", 0)
    if lineish_count == 0:
        return False

    # Ensure no LWPOLYLINE is closed (to avoid polygons)
    for e in entities:
        if e.dxftype() == "LWPOLYLINE" and is_lwpolyline_closed(e):
            return False

    return True

def save_entities_as_dxf(entities, filename):
    """Save given entities to a fresh DXF."""
    new_doc = ezdxf.new(setup=True)
    new_msp = new_doc.modelspace()
    for e in entities:
        try:
            new_msp.add_foreign_entity(e)
        except Exception as ex:
            print(f"  Skipped {e.dxftype()}: {ex}")
    new_doc.saveas(filename)

# === LOAD & EVALUATE ===
doc = ezdxf.readfile(input_dxf)
msp = doc.modelspace()

# Group entities by layer
layer_entities = defaultdict(list)
for ent in msp:
    layer_entities[ent.dxf.layer].append(ent)

# Find candidate layers that satisfy criteria
candidates = []
for layer, ents in layer_entities.items():
    if layer_matches_section_criteria(ents):
        # Score by text count (prefer layer with most section letters)
        tcount = sum(1 for e in ents if e.dxftype() in ("TEXT", "MTEXT"))
        total = len(ents)
        candidates.append((layer, tcount, total, ents))

if not candidates:
    print(" No layer matched the 'section lines + letters' criteria.")
else:
    # Pick the best candidate: highest text count, then most entities as tiebreaker
    candidates.sort(key=lambda x: (x[1], x[2]), reverse=True)
    best_layer, best_texts, best_total, best_entities = candidates[0]

    # Save as section_lines.dxf (overwrite intentionally)
    save_entities_as_dxf(best_entities, out_file)

    # Console report
    print(f" Selected layer: '{best_layer}'")
    print(f"   - Text count: {best_texts}")
    print(f"   - Entity total: {best_total}")
    # Type breakdown for visibility
    type_counts = Counter(e.dxftype() for e in best_entities)
    for t, c in sorted(type_counts.items()):
        print(f"   - {t}: {c}")
    print(f" Saved as: {out_file}")


 Selected layer: 'SECTION LINE NAUKARI LST'
   - Text count: 36
   - Entity total: 55
   - LINE: 5
   - LWPOLYLINE: 14
   - MTEXT: 36
 Saved as: D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer\section_lines.dxf


In [24]:
import ezdxf
import os
from collections import defaultdict

# Input DXF
input_dxf = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\dxf_file\ACW_NLM_Proposed_Pit_and_Dumping_area_for_FY_2024-25.dxf"

# Output folder
output_folder = r"D:\2_Analytics\6_plan_vs_actual\raw_data_dwg_file\split_by_layer_new"
os.makedirs(output_folder, exist_ok=True)

# Load DXF
doc = ezdxf.readfile(input_dxf)
msp = doc.modelspace()

# Group entities by layer name
layer_entities = defaultdict(list)
for entity in msp:
    layer_entities[entity.dxf.layer].append(entity)

# Function to save entities into new DXF
def save_entities(entities, filename):
    if not entities:
        return
    new_doc = ezdxf.new(setup=True)
    new_msp = new_doc.modelspace()
    for e in entities:
        try:
            new_msp.add_foreign_entity(e)
        except Exception as ex:
            print(f" Skipped {e.dxftype()}: {ex}")
    new_doc.saveas(filename)

# Function to extract geometry details
def describe_entity(e):
    etype = e.dxftype()
    if etype == "LINE":
        return f"LINE: start={e.dxf.start}, end={e.dxf.end}"
    elif etype in ["LWPOLYLINE", "POLYLINE"]:
        pts = list(e.get_points("xy"))
        return f"{etype}: vertices={pts}"
    elif etype == "CIRCLE":
        return f"CIRCLE: center={e.dxf.center}, radius={e.dxf.radius}"
    elif etype == "ARC":
        return f"ARC: center={e.dxf.center}, radius={e.dxf.radius}, " \
               f"angles=({e.dxf.start_angle}, {e.dxf.end_angle})"
    elif etype in ["TEXT", "MTEXT"]:
        return f"{etype}: text='{e.dxf.text}', insert={e.dxf.insert}"
    elif etype == "POINT":
        return f"POINT: location={e.dxf.location}"
    elif etype == "HATCH":
        return f"HATCH: pattern={e.dxf.pattern_name}, solid_fill={e.dxf.solid_fill}"
    else:
        return f"{etype}: geometry not detailed"

# Save one DXF per layer & print geometries
for layer, entities in layer_entities.items():
    safe_layer = layer.replace(" ", "_")  # avoid spaces in filenames
    out_file = os.path.join(output_folder, f"{safe_layer}.dxf")
    save_entities(entities, out_file)

    print(f"\n🔹 Layer '{layer}' → {len(entities)} entities")
    for e in entities[:10]:  # print only first 10 per layer to avoid flooding
        print("   ", describe_entity(e))

print("\n Done: Split DXF by layer and printed geometries")



🔹 Layer 'PRO 2024-25 RL216-208' → 6 entities
    HATCH: pattern=ANSI31, solid_fill=0
    LWPOLYLINE: vertices=[(np.float64(308502.2718505859), np.float64(2189320.873657227)), (np.float64(308545.0464212653), np.float64(2189306.451570201)), (np.float64(308691.8540329212), np.float64(2189945.992872612)), (np.float64(308556.7445891708), np.float64(2189972.140751762)), (np.float64(308489.8186633182), np.float64(2189937.203105689)), (np.float64(308478.446248047), np.float64(2189881.108937794)), (np.float64(308293.4234819336), np.float64(2189956.015454346)), (np.float64(308304.9543865915), np.float64(2189993.88547646)), (np.float64(307794.0884599609), np.float64(2190181.320874268)), (np.float64(307652.6556799999), np.float64(2190178.63376)), (np.float64(308586.0487871093), np.float64(2189814.054642731)), (np.float64(308502.2718505859), np.float64(2189320.873657227))]
    TEXT: text='216.000', insert=(308605.2381656518, 2189556.376533808, 0.0)
    TEXT: text='208.000', insert=(308589.07119338